In [ ]:
#!pip install pandas_datareader

In [ ]:
#!pip install yfinance

In [ ]:
#!pip install html5lib

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
import os
import yfinance as yf
from time import sleep
import glob

## Selenium

### Configurando WebDriver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Configurando pasta de download

In [3]:
chromeOptions = webdriver.ChromeOptions()
download_path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data'
prefs = {"download.default_directory" : download_path}
chromeOptions.add_experimental_option("prefs",prefs)

In [4]:
driver = webdriver.Chrome(options=chromeOptions)

In [6]:
os.listdir()
os.chdir('data')

In [7]:
os.listdir()

[]

IBOV: Índice Bovespa - É o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro

IBXX: Índice IBrX 100 - 100 ativos de maior negociabilidade e representatividade do mercado de ações brasileiro.

IBXL: Índice IBrX 50 - 50 ativos de maior negociabilidade e representatividade do mercado de ações brasileiro.

IBRA: Índice Brasil Amplo - O objetivo do IBrA é ser o indicador do desempenho médio das cotações de todos os ativos negociados no mercado a vista (lote-padrão) da B3 que atendam a critérios mínimos de liquidez e presença em pregão, de forma a oferecer uma visão ampla do mercado acionário.

IFNC: Índice Financeiro - ativos de maior negociabilidade e representatividade dos setores de intermediários financeiros, serviços financeiros diversos, previdência e seguros.

ICON: Índice de Consumo - ativos de maior negociabilidade e representatividade dos setores de consumo cíclico, consumo não cíclico e saúde.

IEEX: Índice de Energia Elétrica - ativos de maior negociabilidade e representatividade do setor de energia elétrica.

IFIX: Índice de Fundos Imobiliários - Fundos imobiliários negociados nos mercados de bolsa e de balcão organizado da B3.

IFIL: Índice de Fundos Imobiliários de Alta Liquidez - Fundos imobiliários mais líquidos negociados nos mercados de bolsa e de balcão organizado da B3.

IMAT: Índice de Materiais Básicos - ativos de maior negociabilidade e representatividade do setor de materiais básicos.

IDIV: Índice Dividendos - ativos que se destacaram em termos de remuneração dos investidores, sob a forma de dividendos e juros sobre o capital próprio.

INDX: Índice do Setor Industrial - ativos de maior negociabilidade e representatividade dos setores da atividade industrial compreendidos por materiais básicos, bens industriais, consumo cíclico, consumo não cíclico, tecnologia da informação e saúde.

IMOB: Índice Imobiliário - ativos de maior negociabilidade e representatividade dos setores da atividade imobiliária compreendidos por exploração de imóveis e construção civil.

MLCX: Índice MidLarge Cap - ativos de uma carteira composta pelas empresas de maior capitalização.

SMLL: Índice Small Cap - empresas de menor capitalização.

UTIL: Índice Utilidade Pública - ativos de maior negociabilidade e representatividade do setor de utilidade pública (energia elétrica, água e saneamento e gás).

IVBX: Índice Valor empresas bem conceituadas pelos investidores.

In [ ]:
indice = 'ibov'

In [ ]:
url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'

In [ ]:
driver.get(url)

In [ ]:
driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")

In [ ]:
driver.find_element(By.LINK_TEXT, 'Download').click()

In [ ]:
driver.close()

In [ ]:
os.getcwd()
files = list(glob.glob('*csv'))
print(files)

files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

print(files)

## Download Index file (.csv)

In [24]:
def get_index_stocks(index, wait=10):
    '''
        Recieves the Index name, download a file that contains the index stocks, and return the name of the downloaded file
    '''


    #Chrome WebDriver opens the index webside
    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index.upper()}?language=pt-br'
    driver.get(url)
    driver.implicitly_wait(wait)

    driver.find_element(By.ID, 'segment').send_keys("Setor de Atuação")
    driver.implicitly_wait(wait)
    driver.find_element(By.LINK_TEXT, 'Download').click()
    driver.implicitly_wait(wait)

    #set the directory
    os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data')

    #get the files from the selectec directory and sort them by modification date
    files = files = list(glob.glob('*csv'))
    sleep(wait)
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

    return files[0]

## Get the Index DataFrame

In [13]:
def create_df(file):
    '''Essa é uma função que recebe o nome do índice da Bolsa de Valores e retorna um DataFrame com todas as empresas e seus respectivos setores'''

    #criando DataFrame
    DataFrame = pd.read_csv(file, 
                                encoding='ISO-8859-1',
                                header=1,
                                sep=';',
                                decimal=',',
                                thousands='.',
                                skipfooter=2,
                                engine='python',
                                index_col=False)
    
    #normalizando colunas
    DataFrame.columns = [re.sub('[\.()]', '', re.sub(' ', '_', unidecode(columns.lower()))) for columns in DataFrame.columns]

    return DataFrame

## Creating DataFrames

In [25]:
ibov = create_df(get_index_stocks('ibov'))

In [26]:
ibrx = create_df(get_index_stocks('ibxx'))

In [27]:
ibrx50 = create_df(get_index_stocks('ibxl'))

## Verificando empresas que divergem nos dois índices

In [28]:
emp_ibrx = ibrx['codigo']
set_ibrx = set(emp_ibrx)
len(set_ibrx)

92

In [29]:
emp_ibov = ibov['codigo']
set_ibov = set(emp_ibov)
len(set_ibov)

100

In [20]:
print(len(set_ibov.difference(set_ibrx)))
print(set_ibov.difference(set_ibrx))

0
set()


In [21]:
print(len(set_ibrx.difference(set_ibov)))
print(set_ibrx.difference(set_ibov))

0
set()


## Criando DataFrame unindo os dois índices

In [ ]:
df_empresas = pd.concat([ibrx, ibov]).drop_duplicates(subset='codigo', ignore_index=True).drop(['qtde_teorica',r'part_%',r'part_%acum'], axis=1)

## Criando lista com o código dos pricipais ativos

In [ ]:
codigo = df_empresas['codigo']

### Criando lista com o codigo das ações do Yahoo Finance

In [ ]:
codigo_yfinance = list(codigo+'.SA')
codigo_yfinance

## Yahoo Finance

In [ ]:
yf.Ticker('VALE3.SA').history('20y')

## Função para baixar séries históricas do site da b3 -- Não funciona

In [26]:
'''
url = 'https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)
soup.find_all('select')'''

In [41]:
driver = webdriver.Chrome(options=chromeOptions)

In [54]:
url = f'https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/'
driver.get(url)
driver.implicitly_wait(5)

In [56]:
driver.find_element(By.XPATH, '//*[@id="conteudo-principal"]/div[2]/div/div/ul/li[4]/a').click()

In [ ]:
driver.find_element(By.LINK_TEXT, 'Download').click()
driver.implicitly_wait(wait)

In [ ]:
os.chdir(r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data')

In [ ]:
files = files = list(glob.glob('*txt'))

In [ ]:
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

## Unzip files

In [96]:
import zipfile
path = r'C:\Users\Pedro\OneDrive\Desktop\Ironhack\04. GitHub\data_visualization\data/'

i = 1986

while i < 2000:
    with zipfile.ZipFile(f'{path}COTAHIST_A{i}.zip', 'r') as zip_ref:
        zip_ref.extractall(path)
    i +=1



## Rename files

In [103]:
os.listdir()

['COTAHIST_A1986.txt',
 'COTAHIST_A1987.txt',
 'COTAHIST_A1988.txt',
 'COTAHIST_A1989.txt',
 'COTAHIST_A1990.txt',
 'COTAHIST_A1991.txt',
 'COTAHIST_A1992.txt',
 'COTAHIST_A1993.txt',
 'COTAHIST_A1994.txt',
 'COTAHIST_A1995.txt',
 'COTAHIST_A1996.txt',
 'COTAHIST_A1997.txt',
 'COTAHIST_A1998.txt',
 'COTAHIST_A1999.txt',
 'COTAHIST_A2000.txt',
 'COTAHIST_A2001.txt',
 'COTAHIST_A2002.txt',
 'COTAHIST_A2003.txt',
 'COTAHIST_A2004.txt',
 'COTAHIST_A2005.txt',
 'COTAHIST_A2006.txt',
 'COTAHIST_A2007.txt',
 'COTAHIST_A2008.txt',
 'COTAHIST_A2009.txt',
 'COTAHIST_A2010.txt',
 'COTAHIST_A2011.txt',
 'COTAHIST_A2012.txt',
 'COTAHIST_A2013.txt',
 'COTAHIST_A2014.txt',
 'COTAHIST_A2015.txt',
 'COTAHIST_A2016.txt',
 'COTAHIST_A2017.txt',
 'COTAHIST_A2018.txt',
 'COTAHIST_A2019.txt',
 'COTAHIST_A2020.txt',
 'COTAHIST_A2021.txt',
 'IBOVDia_21-11-22 (1).csv',
 'IBOVDia_21-11-22 (10).csv',
 'IBOVDia_21-11-22 (11).csv',
 'IBOVDia_21-11-22 (2).csv',
 'IBOVDia_21-11-22 (3).csv',
 'IBOVDia_21-11-22 (4).cs

In [102]:
i = 2000

while i < 2022:
    os.rename(f'COTAHIST_A{i}.TXT', f'COTAHIST_A{i}.txt')
    i +=1

## Configurações de leitura das séries históricas da b3

In [ ]:
widths = [2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]
columns = [
"tipo_registro",
"data_pregao",
"cod_bdi",
"cod_negociacao",
"tipo_mercado",
"noma_empresa",
"especificacao_papel",
"prazo_dias_merc_termo",
"moeda_referencia",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"numero_negocios",
"quantidade_papeis_negociados",
"volume_total_negociado",
"preco_exercicio",
"ìndicador_correcao_precos",
"data_vencimento" ,
"fator_cotacao",
"preco_exercicio_pontos",
"codigo_isin",
"num_distribuicao_papel"]

In [62]:
df = pd.read_fwf('COTAHIST_A2019.TXT',
                                encoding='ISO-8859-1',
                                header=0,
                                widths=widths,
                                skipfooter=1,
                                engine='python',
                                parse_dates=[1],
                                infer_datetime_format=True,
                                index_col=False)

df.columns = columns

,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,preco_abertura,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel


In [89]:
df.head(50)

,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,preco_abertura,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
0,1,2019-01-02,2,AALR3,10,ALLIAR,ON NM,NaN,R$,1331,...,856,264200,352396200,0,0,99991231,1,0,BRAALRACNOR6,100
1,1,2019-01-02,2,AAPL34,10,APPLE,DRN,NaN,R$,6071,...,45,16800,101596900,0,0,99991231,1,0,BRAAPLBDR004,127
2,1,2019-01-02,2,ABBV34,10,ABBVIE,DRN,NaN,R$,34284,...,1,300,10285200,0,0,99991231,1,0,BRABBVBDR001,111
3,1,2019-01-02,2,ABCB4,10,ABC BRASIL,PN EJS N2,NaN,R$,1647,...,3477,571700,959830000,0,0,99991231,1,0,BRABCBACNPR4,131
4,1,2019-01-02,2,ABEV3,10,AMBEV S/A,ON EJ,NaN,R$,1540,...,45941,18692900,29820195400,0,0,99991231,1,0,BRABEVACNOR1,121
5,1,2019-01-02,2,ADHM3,10,ADVANCED-DH,ON,NaN,R$,149,...,3,1800,269800,0,0,99991231,1,0,BRADHMACNOR9,101
6,1,2019-01-02,2,AFLT3,10,AFLUENTE T,ON,NaN,R$,548,...,4,500,271600,0,0,99991231,1,0,BRAFLTACNOR1,113
7,1,2019-01-02,2,AGRO3,10,BRASILAGRO,ON NM,NaN,R$,1580,...,165,30000,47598500,0,0,99991231,1,0,BRAGROACNOR7,108
8,1,2019-01-02,2,ALPA3,10,ALPARGATAS,ON N1,NaN,R$,1620,...,21,5500,9084800,0,0,99991231,1,0,BRALPAACNOR0,227
9,1,2019-01-02,2,ALPA4,10,ALPARGATAS,PN N1,NaN,R$,1703,...,1825,432100,747451000,0,0,99991231,1,0,BRALPAACNPR7,227


In [77]:
df[(df['tipo_mercado']==70)].head(50)

,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,preco_abertura,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
231896,1,2019-01-02,78,BBASA510,70,BBASE,ON NM,0.0,R$,75,...,9,14200,1011200,5096,0,20190121,1,0,BRBBASACNOR3,280
231897,1,2019-01-02,78,BBASB512,70,BBASE,ON NM,0.0,R$,70,...,3,2100,200000,5114,0,20190218,1,0,BRBBASACNOR3,280
231898,1,2019-01-02,78,BBASA500,70,BBASE,ON NM,0.0,R$,33,...,3,3100,189000,4996,0,20190121,1,0,BRBBASACNOR3,280
231899,1,2019-01-02,78,BBASA455,70,BBASE,ON NM,0.0,R$,238,...,17,4600,1710700,4546,0,20190121,1,0,BRBBASACNOR3,280
231900,1,2019-01-02,78,BBASA450,70,BBASE,ON NM,0.0,R$,222,...,5,2100,914200,4496,0,20190121,1,0,BRBBASACNOR3,280
231901,1,2019-01-02,78,BBASA470,70,BBASE,ON NM,0.0,R$,112,...,2,600,84300,4696,0,20190121,1,0,BRBBASACNOR3,280
231906,1,2019-01-02,78,BRKMA50,70,BRKM FM,PNA N1,0.0,R$,50,...,1,700,35000,5000,0,20190121,1,0,BRBRKMACNPA4,116
231907,1,2019-01-02,78,BRKMA48,70,BRKM FM,PNA N1,0.0,R$,110,...,2,200,21000,4800,0,20190121,1,0,BRBRKMACNPA4,116
231916,1,2019-01-02,78,BRFSD232,70,BRFSE,ON NM,0.0,R$,115,...,4,1300,156500,2325,0,20190415,1,0,BRBRFSACNOR8,118
231917,1,2019-01-02,78,BRFSD240,70,BRFS,ON NM,0.0,R$,120,...,1,200,24000,2400,0,20190415,1,0,BRBRFSACNOR8,118


In [76]:
df[(df['data_pregao']=='2019-01-02') & (df['tipo_mercado']==10)]['cod_negociacao'].unique()

array(['AALR3', 'AAPL34', 'ABBV34', 'ABCB4', 'ABEV3', 'ADHM3', 'AFLT3',
       'AGRO3', 'ALPA3', 'ALPA4', 'ALSC3', 'ALUP3', 'ALUP4', 'ALUP11',
       'AMAR3', 'AMGN34', 'AMZO34', 'ANIM3', 'APER3', 'ARZZ3', 'ATOM3',
       'ATTB34', 'AZEV4', 'AZUL4', 'B3SA3', 'BAHI3', 'BAUH4', 'BAZA3',
       'BBAS3', 'BBDC3', 'BBDC4', 'BBRK3', 'BBSE3', 'BEEF3', 'BEES3',
       'BEES4', 'BERK34', 'BGIP4', 'BIDI4', 'BIIB34', 'BIOM3', 'BKBR3',
       'BMEB4', 'BNBR3', 'BOAC34', 'BOBR4', 'BOEI34', 'BOXP34', 'BPAC3',
       'BPAC5', 'BPAC11', 'BPAN4', 'BRAP3', 'BRAP4', 'BRDT3', 'BRFS3',
       'BRIV4', 'BRKM3', 'BRKM5', 'BRML3', 'BRPR3', 'BRSR3', 'BRSR6',
       'BSEV3', 'BTOW3', 'BTTL3', 'CAMB4', 'CAML3', 'CARD3', 'CBEE3',
       'CCPR3', 'CCRO3', 'CCXC3', 'CEDO4', 'CEEB3', 'CELP3', 'CELP5',
       'CELP7', 'CESP3', 'CESP5', 'CESP6', 'CGAS3', 'CGAS5', 'CGRA3',
       'CGRA4', 'CHVX34', 'CIEL3', 'CLGN34', 'CLSC4', 'CMCS34', 'CMIG3',
       'CMIG4', 'COCA34', 'COCE3', 'COCE5', 'COLG34', 'COPH34', 'COWC34',
 

In [59]:

os.chdir('data')

In [60]:
os.listdir()

['COTAHIST_A2015.ZIP',
 'COTAHIST_A2019.TXT',
 'COTAHIST_A2019.ZIP',
 'DemoCotacoesHistoricas12022003.txt',
 'IBOVDia_21-11-22 (1).csv',
 'IBOVDia_21-11-22 (2).csv',
 'IBOVDia_21-11-22 (3).csv',
 'IBOVDia_21-11-22 (4).csv',
 'IBOVDia_21-11-22 (5).csv',
 'IBOVDia_21-11-22 (6).csv',
 'IBOVDia_21-11-22 (7).csv',
 'IBOVDia_21-11-22 (8).csv',
 'IBOVDia_21-11-22 (9).csv',
 'IBOVDia_21-11-22.csv',
 'ibov_empresas.csv',
 'ibov_setor.csv',
 'ibrx100_setor.csv',
 'ibrx_empresas.csv',
 'IBXXDia_21-11-22 (1).csv',
 'IBXXDia_21-11-22 (2).csv',
 'IBXXDia_21-11-22.csv',
 'SeriesHistoricas_DemoCotacoesHistoricas12022003.zip',
 'SeriesHistoricas_Layout.pdf']